In [1]:
import os
import sys

sys.path.append(os.getcwd().split('notebooks')[0])

from src.utilities.utils import *

Device: cpu


In [2]:
from src.utilities.MLP_grid_search import *

import torch
from torch import nn
import pandas as pd
import numpy as np

In [2]:
# select device to run this on
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# file = "MD-HIT_trunc_5796810_formation_energy_per_atom_1_0113_221732.pt"
# load_path = f"/Users/shaanpakala/Desktop/Vagelis_Research/materials_design/training_tensors/{file}"

# sparse_components = torch.load(load_path)
# sparse_tensor = torch.sparse_coo_tensor(
#     sparse_components['indices'],
#     sparse_components['values'],
#     sparse_components['size']
# ).coalesce()

# x = np.array(sparse_tensor.indices().t())
# y = np.array(sparse_tensor.values().unsqueeze(1))

In [4]:
if False:
    
    data = pd.read_csv("/Users/shaanpakala/Desktop/NSF_REU_2024/Research/classification_datasets/glass.data",
                       names = [f"Feature_{i+1}" for i in range(9)] + ['Target'])
    
    unique_classes = list(set(data['Target']))
    unique_classes = {unique_classes[i]:i for i in range(len(unique_classes))}

    data['Target'] = data['Target'].map(lambda x: unique_classes[x])
    data = np.array(data)

    x = data[:, :-1]
    y = data[:, -1]

    del data
    
elif False:
    
    train = pd.read_csv("/Users/shaanpakala/Desktop/Lonardi_Research/panCleave/data/panCleave_training_data.csv")
    test = pd.read_csv("/Users/shaanpakala/Desktop/Lonardi_Research/panCleave/data/panCleave_test_data.csv")
    
    data = pd.concat([train, test]).reset_index(drop = True)
    
    del train, test
    
    data['Ordinal'] = data['Ordinal'].map(lambda x: [int(y) for y in x[1:-1].split(',')])
    
    def str2int(x):
        nums = '0123456789'
        nums = {i:True for i in nums}
        return int(''.join([y for y in x if y in nums]))

    convert_df_col = lambda x: np.array([[str2int(z) for z in y.split()] for y in x.split('],')])

    vector_types = ['FP-scale', 'Z-scale', 'ST-scale']

    for col in vector_types: data[col] = data[col].map(convert_df_col)

    master_dict = dict()

    for vector_type in vector_types:
        
        current_vector_dict = dict()
        conflicts = 0
        for i in range(data.shape[0]):
            
            sample = data.iloc[i]
            
            for idx in range(8):
                
                c = sample['Ordinal'][idx]
                mapping = sample[vector_type][idx]
                
                if c not in current_vector_dict:
                    current_vector_dict[c] = list(mapping)
                    
                elif (current_vector_dict[c] - np.array(mapping)).sum() != 0: conflicts += 1
            
        if conflicts > 0: print(f"{vector_type} had {conflicts} conflicts.")
        master_dict[vector_type] = current_vector_dict
        del current_vector_dict
        
    unique_values = list(set(data['Label']))
    unique_values = {unique_values[i]:i for i in range(len(unique_values))}

    y =  np.array(list(data['Label'].map(lambda x: unique_values[x])))
    x = np.array(list(data['Ordinal']))
    
elif False:
    
    data = np.array(pd.read_csv('/Users/shaanpakala/Desktop/NSF_REU_2024/Research/classification_datasets/dermatology_database.csv'))

    x = np.array(data[:, :-2], dtype = np.float32)
    y = np.array(data[:, -1], dtype = np.int32)
    
    del data
    
    l = list(set(y))
    l = {l[i]:i for i in range(len(l))}

    y = np.array([l[i] for i in y], dtype = np.int32)
    del l

elif True:

    data = np.load("/Users/shaanpakala/Desktop/NSF_REU_2024/Research/classification_datasets/MiniBoone_Particle_ID.npy")

    x = data[:, :-1]
    y = data[:, -1]
    
    del data    

In [5]:
print(f"X Shape: {x.shape}\nY Shape: {y.shape}")

X Shape: (130064, 50)
Y Shape: (130064,)


In [6]:
# input dictionary corresponding to parameters & ranges to test

param_dict = {

              # for hyperparameters to be tuned, enter list of values

              'hidden_dims':[
                             [64], [128], [256], [512], [1024],
                             [20, 20], [32, 20], [64, 64], [128, 128], [256, 256], [512, 512],
                             [20, 32, 32], [32, 64, 64], [128, 128, 128],
                             [256, 256, 256], [512, 512, 512], 
                             [64 for _ in range(4)], [128 for _ in range(4)],
                             [512 for _ in range(4)],
                             ],

              'dropouts':[0, 0.05, 0.1, 0.2],
              'num_epochs':[5, 10, 15, 20, 30, 50, 100],
              'batch_size':[16, 32, 64, 128, 256, 512],
              'lr': [1e-3, 1e-2, 1e-1],
              'wd': [1e-4, 1e-3, 1e-2, 1e-1],
              'activation':['relu', 'tanh'],
              'last_act':['softmax', 'sigmoid'],

            #   'change_input':[None,
            #                   master_dict['FP-scale'], 
            #                   master_dict['Z-scale'], 
            #                   master_dict['ST-scale']],


              # for constant hyperparameters, enter one value

              'loss_fn':'cross_entropy',
              'stop_early':True
              }

In [7]:
# run function to approximate best parameter combinations using Sparse Tensor Completion

best_estimated_params = return_best_k_params(hyperameter_dict = param_dict,              # dictionary of parameters
                                             X = x,
                                             Y = y,
                                             num_top_combinations = 15,                  # number of best estimated combinations to return
                                             cv_splits = 5,                           # number of Cross Validation folds
                                             num_training_entries = 500,              
                                             tensor_completion_model = 'cpd.smooth',     # sparse tensor completion model
                                             task_type = 'classification',
                                             metric = 'accuracy',
                                             rank = 20,                                  # rank decomposition used for tensor completion
                                             device = device,
                                             verbose = True)

500/153216 total combinations in sparse tensor.

1/500 param_combinations done.
2/500 param_combinations done.
3/500 param_combinations done.
4/500 param_combinations done.
5/500 param_combinations done.
6/500 param_combinations done.
7/500 param_combinations done.
8/500 param_combinations done.
9/500 param_combinations done.
10/500 param_combinations done.
11/500 param_combinations done.
12/500 param_combinations done.
13/500 param_combinations done.
14/500 param_combinations done.
15/500 param_combinations done.
16/500 param_combinations done.
17/500 param_combinations done.
18/500 param_combinations done.
19/500 param_combinations done.
20/500 param_combinations done.
21/500 param_combinations done.
22/500 param_combinations done.
23/500 param_combinations done.
24/500 param_combinations done.
25/500 param_combinations done.
26/500 param_combinations done.
27/500 param_combinations done.
28/500 param_combinations done.
29/500 param_combinations done.
30/500 param_combinations done.


In [8]:
for i in range(len(best_estimated_params)):
    best_estimated_params[i][0]['Evaluation'] = best_estimated_params[i][1]

hyperparams = list(best_estimated_params[0][0])

In [9]:
pd.DataFrame(np.array([[str(param_set[0][p]) for p in hyperparams] for param_set in best_estimated_params]),
             columns = hyperparams)

,hidden_dims,dropouts,num_epochs,batch_size,lr,wd,activation,last_act,Evaluation
0,"[20, 32, 32]",0,100,32,0.01,0.0001,relu,sigmoid,0.782119150944364
1,[512],0.05,10,16,0.01,0.001,tanh,softmax,0.7360422886337755
2,[128],0.1,15,16,0.01,0.001,relu,softmax,0.7194017830578724
3,"[128, 128, 128, 128]",0,5,128,0.1,0.1,relu,sigmoid,0.7193710053040329
4,"[512, 512, 512, 512]",0,10,256,0.01,0.001,relu,softmax,0.719360746052753
5,[1024],0,20,32,0.1,0.0001,tanh,softmax,0.7159418505637459
6,"[512, 512]",0.05,100,64,0.001,0.001,tanh,softmax,0.6958337180552564
7,"[256, 256]",0.1,30,128,0.01,0.001,tanh,softmax,0.6769259179465082
8,"[20, 32, 32]",0.2,5,256,0.01,0.01,tanh,sigmoid,0.6499107445138653
9,[256],0.2,5,128,0.1,0.001,tanh,softmax,0.6470381541555097
